# Context-Conditional GAN

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import os

In [2]:
from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Flatten, Dropout
from keras.layers import BatchNormalization
from keras.layers import Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import to_categorical

Using TensorFlow backend.


# Generator

In [3]:
def conv2d(layer_input, filters, f_size=4, bn=True):
    """Layers used during downsampling"""
    d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
    d = LeakyReLU(alpha=0.2)(d)
    if bn:
        d = BatchNormalization(momentum=0.8)(d)
    return d

def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
    """Layers used during upsampling"""
    u = UpSampling2D(size=2)(layer_input)
    u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
    if dropout_rate:
        u = Dropout(dropout_rate)(u)
    u = BatchNormalization(momentum=0.8)(u)
    u = Concatenate()([u, skip_input])
    return u

In [4]:
def build_generator(img_shape, gf, channels):
    """U-Net Generator"""
    img = Input(shape=img_shape)

    # Downsampling
    d1 = conv2d(img, gf, bn=False)
    d2 = conv2d(d1, gf*2)
    d3 = conv2d(d2, gf*4)
    d4 = conv2d(d3, gf*8)

    # Upsampling
    u1 = deconv2d(d4, d3, gf*4)
    u2 = deconv2d(u1, d2, gf*2)
    u3 = deconv2d(u2, d1, gf)

    u4 = UpSampling2D(size=2)(u3)
    output_img = Conv2D(channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u4)

    return Model(img, output_img)

# Discriminator

In [5]:
def build_discriminator(img_shape, df, num_classes):

    img = Input(shape=img_shape)

    model = Sequential()
    model.add(Conv2D(df, kernel_size=4, strides=2, padding='same', input_shape=img_shape))
    model.add(LeakyReLU(alpha=0.8))
    model.add(Conv2D(df * 2, kernel_size=4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(InstanceNormalization())
    model.add(Conv2D(df * 4, kernel_size=4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(InstanceNormalization())

    model.summary()

    img = Input(shape=img_shape)
    features = model(img)

    validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(features)

    label = Flatten()(features)
    label = Dense(num_classes+1, activation="softmax")(label)

    return Model(img, [validity, label])

# Helper functions

In [6]:
def save(model, model_name):
    model_path = "saved_model/%s.json" % model_name
    weights_path = "saved_model/%s_weights.hdf5" % model_name
    options = {"file_arch": model_path,
                "file_weight": weights_path}
    json_string = model.to_json()
    open(options['file_arch'], 'w').write(json_string)
    model.save_weights(options['file_weight'])

def save_model(G, D):
    save(G, "ccgan_generator")
    save(D, "ccgan_discriminator")

In [7]:
def mask_randomly(imgs, img_rows, mask_height):
    y1 = np.random.randint(0, img_rows - mask_height, imgs.shape[0])
    y2 = y1 + mask_height
    x1 = np.random.randint(0, img_rows - mask_width, imgs.shape[0])
    x2 = x1 + mask_width

    masked_imgs = np.empty_like(imgs)
    for i, img in enumerate(imgs):
        masked_img = img.copy()
        _y1, _y2, _x1, _x2 = y1[i], y2[i], x1[i], x2[i],
        masked_img[_y1:_y2, _x1:_x2, :] = 0
        masked_imgs[i] = masked_img

    return masked_imgs

In [8]:
def sample_images(G, epoch, 
                  imgs, img_rows, mask_height):
    r, c = 3, 6

    masked_imgs = mask_randomly(imgs, img_rows, mask_height)
    gen_imgs = G.predict(masked_imgs)

    imgs = (imgs + 1.0) * 0.5
    masked_imgs = (masked_imgs + 1.0) * 0.5
    gen_imgs = (gen_imgs + 1.0) * 0.5

    gen_imgs = np.where(gen_imgs < 0, 0, gen_imgs)

    fig, axs = plt.subplots(r, c)
    for i in range(c):
        axs[0,i].imshow(imgs[i, :, :, 0], cmap='gray')
        axs[0,i].axis('off')
        axs[1,i].imshow(masked_imgs[i, :, :, 0], cmap='gray')
        axs[1,i].axis('off')
        axs[2,i].imshow(gen_imgs[i, :, :, 0], cmap='gray')
        axs[2,i].axis('off')
    fig.savefig("images/%d.png" % epoch)
    plt.close()

In [9]:
def imresize(img, size):
    """workaroung imresize"""
    from PIL import Image
    im = Image.fromarray(np.asarray(img).astype(np.uint8))
    new_image = np.array(im.resize(size, Image.BICUBIC))
    return new_image

In [10]:
def train(G, D, combined,
          img_rows, mask_height,
          num_classes,
          epochs, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, y_train), (_, _) = mnist.load_data()

    # Rescale MNIST to 32x32
    X_train = np.array([imresize(x, [img_rows, img_cols]) for x in X_train])

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)
    y_train = y_train.reshape(-1, 1)

    # Adversarial ground truths
    valid = np.ones((batch_size, 4, 4, 1))
    fake = np.zeros((batch_size, 4, 4, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Sample half batch of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        labels = y_train[idx]

        masked_imgs = mask_randomly(imgs, img_rows, mask_height)

        # Generate a half batch of new images
        gen_imgs = G.predict(masked_imgs)

        # One-hot encoding of labels
        labels = to_categorical(labels, num_classes=num_classes+1)
        fake_labels = to_categorical(np.full((batch_size, 1), num_classes), num_classes=num_classes+1)

        # Train the discriminator
        d_loss_real = D.train_on_batch(imgs, [valid, labels])
        d_loss_fake = D.train_on_batch(gen_imgs, [fake, fake_labels])
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------

        # Train the generator
        g_loss = combined.train_on_batch(masked_imgs, valid)

        # Plot the progress
        print ("%d [D loss: %f, op_acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[4], g_loss))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0 or epoch == epochs - 1:
            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], 6)
            imgs = X_train[idx]
            sample_images(G, epoch, imgs, img_rows, mask_height)
            save_model(G, D)


# main()

In [11]:
for d in ['saved_model', 'images']:
    if not os.path.exists(d):
        os.makedirs(d)

In [12]:
img_rows = 32
img_cols = 32
channels = 1
img_shape = (img_rows, img_cols, channels)
mask_height = 10
mask_width = 10
num_classes = 10

In [13]:
# Number of filters in first layer of generator and discriminator
gf = 32
df = 64

In [14]:
# create optimizer
optimizer = Adam(0.0002, 0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [15]:
# Build and compile the discriminator
D = build_discriminator(img_shape, df, num_classes)
D.compile(loss=['mse', 'categorical_crossentropy'],
    loss_weights=[0.5, 0.5],
    optimizer=optimizer,
    metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 64)        1088      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 128)         131200    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
instance_normalization_1 (In (None, 8, 8, 128)         2         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 256)         524544    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 4, 4, 256)        

In [16]:
# Build the generator
G = build_generator(img_shape, gf, channels)

In [17]:
# The generator takes noise as input and generates imgs
masked_img = Input(shape=img_shape)
gen_img = G(masked_img)

In [18]:
# For the combined model we will only train the generator
D.trainable = False

In [19]:
# The valid takes generated images as input and determines validity
valid, _ = D(gen_img)

In [20]:
# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator
combined = Model(masked_img , valid)
combined.compile(loss=['mse'],
                 optimizer=optimizer)

In [21]:
## train the model

In [22]:
epochs=20000
epochs=4000
train(G, D, combined, 
      img_rows, mask_height,
      num_classes,
      epochs=epochs, batch_size=32, sample_interval=200)

c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 2.676828, op_acc: 31.25%] [G loss: 1.671727]
1 [D loss: 2.947555, op_acc: 54.69%] [G loss: 1.411595]
2 [D loss: 1.220675, op_acc: 56.25%] [G loss: 0.926958]
3 [D loss: 0.798387, op_acc: 59.38%] [G loss: 1.037243]
4 [D loss: 0.999775, op_acc: 67.19%] [G loss: 0.807718]
5 [D loss: 1.080346, op_acc: 73.44%] [G loss: 0.588819]
6 [D loss: 0.899316, op_acc: 64.06%] [G loss: 0.444817]
7 [D loss: 0.737623, op_acc: 71.88%] [G loss: 0.962830]
8 [D loss: 0.738256, op_acc: 68.75%] [G loss: 0.419260]
9 [D loss: 0.568055, op_acc: 71.88%] [G loss: 0.502009]
10 [D loss: 0.507391, op_acc: 78.12%] [G loss: 0.508464]
11 [D loss: 0.441194, op_acc: 76.56%] [G loss: 0.424369]
12 [D loss: 0.492726, op_acc: 78.12%] [G loss: 0.655686]
13 [D loss: 0.690747, op_acc: 85.94%] [G loss: 0.862430]
14 [D loss: 0.855289, op_acc: 73.44%] [G loss: 0.445141]
15 [D loss: 0.617780, op_acc: 75.00%] [G loss: 0.642001]
16 [D loss: 0.573249, op_acc: 78.12%] [G loss: 1.003126]
17 [D loss: 0.416929, op_acc: 84.38%] [G 

144 [D loss: 0.549511, op_acc: 75.00%] [G loss: 0.403488]
145 [D loss: 0.495174, op_acc: 81.25%] [G loss: 0.442974]
146 [D loss: 0.459040, op_acc: 79.69%] [G loss: 0.349668]
147 [D loss: 0.565326, op_acc: 70.31%] [G loss: 0.478392]
148 [D loss: 0.612672, op_acc: 70.31%] [G loss: 0.422355]
149 [D loss: 0.499508, op_acc: 81.25%] [G loss: 0.383089]
150 [D loss: 0.662975, op_acc: 64.06%] [G loss: 0.436297]
151 [D loss: 0.567225, op_acc: 67.19%] [G loss: 0.339511]
152 [D loss: 0.480788, op_acc: 71.88%] [G loss: 0.432686]
153 [D loss: 0.527470, op_acc: 70.31%] [G loss: 0.469522]
154 [D loss: 0.579658, op_acc: 68.75%] [G loss: 0.411361]
155 [D loss: 0.562804, op_acc: 71.88%] [G loss: 0.365664]
156 [D loss: 0.549312, op_acc: 68.75%] [G loss: 0.426484]
157 [D loss: 0.614819, op_acc: 62.50%] [G loss: 0.398174]
158 [D loss: 0.527271, op_acc: 78.12%] [G loss: 0.408960]
159 [D loss: 0.583362, op_acc: 68.75%] [G loss: 0.434061]
160 [D loss: 0.541418, op_acc: 73.44%] [G loss: 0.386245]
161 [D loss: 0

286 [D loss: 0.534046, op_acc: 71.88%] [G loss: 0.372910]
287 [D loss: 0.537879, op_acc: 67.19%] [G loss: 0.359787]
288 [D loss: 0.489184, op_acc: 71.88%] [G loss: 0.323519]
289 [D loss: 0.370990, op_acc: 89.06%] [G loss: 0.386663]
290 [D loss: 0.498720, op_acc: 71.88%] [G loss: 0.399097]
291 [D loss: 0.393178, op_acc: 82.81%] [G loss: 0.349669]
292 [D loss: 0.476386, op_acc: 70.31%] [G loss: 0.426485]
293 [D loss: 0.394634, op_acc: 81.25%] [G loss: 0.397008]
294 [D loss: 0.406294, op_acc: 82.81%] [G loss: 0.332845]
295 [D loss: 0.480516, op_acc: 79.69%] [G loss: 0.382449]
296 [D loss: 0.561146, op_acc: 57.81%] [G loss: 0.346764]
297 [D loss: 0.437053, op_acc: 79.69%] [G loss: 0.339082]
298 [D loss: 0.430487, op_acc: 78.12%] [G loss: 0.324237]
299 [D loss: 0.477995, op_acc: 75.00%] [G loss: 0.377062]
300 [D loss: 0.585205, op_acc: 73.44%] [G loss: 0.333932]
301 [D loss: 0.419304, op_acc: 78.12%] [G loss: 0.379560]
302 [D loss: 0.418406, op_acc: 81.25%] [G loss: 0.359296]
303 [D loss: 0

428 [D loss: 0.385368, op_acc: 89.06%] [G loss: 0.361320]
429 [D loss: 0.412907, op_acc: 79.69%] [G loss: 0.357937]
430 [D loss: 0.447280, op_acc: 76.56%] [G loss: 0.396408]
431 [D loss: 0.373880, op_acc: 79.69%] [G loss: 0.377474]
432 [D loss: 0.349013, op_acc: 89.06%] [G loss: 0.343092]
433 [D loss: 0.493239, op_acc: 81.25%] [G loss: 0.377026]
434 [D loss: 0.348400, op_acc: 89.06%] [G loss: 0.377136]
435 [D loss: 0.471647, op_acc: 73.44%] [G loss: 0.346729]
436 [D loss: 0.422575, op_acc: 75.00%] [G loss: 0.345771]
437 [D loss: 0.330769, op_acc: 87.50%] [G loss: 0.350416]
438 [D loss: 0.450146, op_acc: 79.69%] [G loss: 0.336176]
439 [D loss: 0.444957, op_acc: 76.56%] [G loss: 0.391175]
440 [D loss: 0.448035, op_acc: 76.56%] [G loss: 0.374538]
441 [D loss: 0.462653, op_acc: 76.56%] [G loss: 0.294840]
442 [D loss: 0.439095, op_acc: 81.25%] [G loss: 0.370970]
443 [D loss: 0.503340, op_acc: 73.44%] [G loss: 0.334834]
444 [D loss: 0.371364, op_acc: 85.94%] [G loss: 0.334781]
445 [D loss: 0

...

3359 [D loss: 0.118728, op_acc: 93.75%] [G loss: 0.882847]
3360 [D loss: 0.136049, op_acc: 93.75%] [G loss: 0.956058]
3361 [D loss: 0.127568, op_acc: 95.31%] [G loss: 1.322439]
3362 [D loss: 0.213335, op_acc: 90.62%] [G loss: 1.695370]
3363 [D loss: 0.348036, op_acc: 89.06%] [G loss: 0.823333]
3364 [D loss: 0.276624, op_acc: 81.25%] [G loss: 1.036168]
3365 [D loss: 0.133599, op_acc: 93.75%] [G loss: 0.973026]
3366 [D loss: 0.109827, op_acc: 98.44%] [G loss: 0.980147]
3367 [D loss: 0.152249, op_acc: 93.75%] [G loss: 0.916357]
3368 [D loss: 0.154999, op_acc: 93.75%] [G loss: 1.004775]
3369 [D loss: 0.087007, op_acc: 98.44%] [G loss: 0.979680]
3370 [D loss: 0.089674, op_acc: 98.44%] [G loss: 0.961507]
3371 [D loss: 0.084673, op_acc: 96.88%] [G loss: 0.980441]
3372 [D loss: 0.129849, op_acc: 93.75%] [G loss: 1.129056]
3373 [D loss: 0.158043, op_acc: 90.62%] [G loss: 0.908294]
3374 [D loss: 0.125929, op_acc: 93.75%] [G loss: 1.055973]
3375 [D loss: 0.141784, op_acc: 93.75%] [G loss: 0.97663

3498 [D loss: 0.120952, op_acc: 92.19%] [G loss: 0.850019]
3499 [D loss: 0.138957, op_acc: 93.75%] [G loss: 1.011218]
3500 [D loss: 0.084001, op_acc: 96.88%] [G loss: 0.944696]
3501 [D loss: 0.083314, op_acc: 96.88%] [G loss: 0.927735]
3502 [D loss: 0.142186, op_acc: 93.75%] [G loss: 1.079179]
3503 [D loss: 0.116609, op_acc: 95.31%] [G loss: 1.014875]
3504 [D loss: 0.103763, op_acc: 95.31%] [G loss: 1.006398]
3505 [D loss: 0.082784, op_acc: 95.31%] [G loss: 1.095914]
3506 [D loss: 0.129256, op_acc: 95.31%] [G loss: 0.980795]
3507 [D loss: 0.083493, op_acc: 96.88%] [G loss: 0.989117]
3508 [D loss: 0.128011, op_acc: 93.75%] [G loss: 0.916063]
3509 [D loss: 0.078354, op_acc: 98.44%] [G loss: 0.912098]
3510 [D loss: 0.061242, op_acc: 100.00%] [G loss: 1.065850]
3511 [D loss: 0.089920, op_acc: 95.31%] [G loss: 1.100992]
3512 [D loss: 0.114521, op_acc: 93.75%] [G loss: 0.970402]
3513 [D loss: 0.087200, op_acc: 96.88%] [G loss: 1.021401]
3514 [D loss: 0.099302, op_acc: 96.88%] [G loss: 0.9879

3637 [D loss: 0.155790, op_acc: 90.62%] [G loss: 0.962881]
3638 [D loss: 0.118955, op_acc: 96.88%] [G loss: 1.236166]
3639 [D loss: 0.120146, op_acc: 96.88%] [G loss: 0.827719]
3640 [D loss: 0.066622, op_acc: 96.88%] [G loss: 1.000134]
3641 [D loss: 0.095652, op_acc: 100.00%] [G loss: 0.854200]
3642 [D loss: 0.054796, op_acc: 100.00%] [G loss: 0.987556]
3643 [D loss: 0.174207, op_acc: 92.19%] [G loss: 0.812392]
3644 [D loss: 0.099114, op_acc: 98.44%] [G loss: 1.060016]
3645 [D loss: 0.102676, op_acc: 95.31%] [G loss: 1.010129]
3646 [D loss: 0.126607, op_acc: 92.19%] [G loss: 0.916692]
3647 [D loss: 0.067317, op_acc: 98.44%] [G loss: 0.968157]
3648 [D loss: 0.060599, op_acc: 100.00%] [G loss: 0.992798]
3649 [D loss: 0.118864, op_acc: 93.75%] [G loss: 0.935110]
3650 [D loss: 0.106938, op_acc: 96.88%] [G loss: 0.914100]
3651 [D loss: 0.110055, op_acc: 95.31%] [G loss: 1.012881]
3652 [D loss: 0.120821, op_acc: 93.75%] [G loss: 1.037257]
3653 [D loss: 0.149548, op_acc: 92.19%] [G loss: 1.00

3776 [D loss: 0.108793, op_acc: 98.44%] [G loss: 0.839405]
3777 [D loss: 0.099631, op_acc: 96.88%] [G loss: 1.027060]
3778 [D loss: 0.062719, op_acc: 100.00%] [G loss: 0.915667]
3779 [D loss: 0.086309, op_acc: 98.44%] [G loss: 0.925586]
3780 [D loss: 0.062194, op_acc: 100.00%] [G loss: 0.970382]
3781 [D loss: 0.054387, op_acc: 98.44%] [G loss: 1.000214]
3782 [D loss: 0.076614, op_acc: 100.00%] [G loss: 0.862091]
3783 [D loss: 0.075956, op_acc: 96.88%] [G loss: 1.018990]
3784 [D loss: 0.110283, op_acc: 98.44%] [G loss: 0.844748]
3785 [D loss: 0.110185, op_acc: 95.31%] [G loss: 0.839612]
3786 [D loss: 0.068211, op_acc: 100.00%] [G loss: 0.956404]
3787 [D loss: 0.115196, op_acc: 96.88%] [G loss: 0.901781]
3788 [D loss: 0.106945, op_acc: 96.88%] [G loss: 0.914702]
3789 [D loss: 0.152510, op_acc: 90.62%] [G loss: 1.440299]
3790 [D loss: 0.077243, op_acc: 96.88%] [G loss: 0.955171]
3791 [D loss: 0.098409, op_acc: 95.31%] [G loss: 0.724387]
3792 [D loss: 0.131078, op_acc: 93.75%] [G loss: 0.9

3915 [D loss: 0.103589, op_acc: 96.88%] [G loss: 0.970987]
3916 [D loss: 0.152206, op_acc: 92.19%] [G loss: 1.026300]
3917 [D loss: 0.069459, op_acc: 98.44%] [G loss: 1.008637]
3918 [D loss: 0.061459, op_acc: 98.44%] [G loss: 1.000216]
3919 [D loss: 0.128235, op_acc: 95.31%] [G loss: 1.093185]
3920 [D loss: 0.178168, op_acc: 89.06%] [G loss: 1.120914]
3921 [D loss: 0.054458, op_acc: 98.44%] [G loss: 1.001776]
3922 [D loss: 0.128991, op_acc: 95.31%] [G loss: 1.033904]
3923 [D loss: 0.109730, op_acc: 93.75%] [G loss: 1.054137]
3924 [D loss: 0.123534, op_acc: 96.88%] [G loss: 0.766407]
3925 [D loss: 0.151191, op_acc: 87.50%] [G loss: 1.300678]
3926 [D loss: 0.225160, op_acc: 90.62%] [G loss: 0.965813]
3927 [D loss: 0.145027, op_acc: 92.19%] [G loss: 0.898567]
3928 [D loss: 0.067085, op_acc: 95.31%] [G loss: 0.994376]
3929 [D loss: 0.051034, op_acc: 98.44%] [G loss: 0.908908]
3930 [D loss: 0.109697, op_acc: 95.31%] [G loss: 0.873783]
3931 [D loss: 0.144584, op_acc: 95.31%] [G loss: 0.97915